# School Timetable Generator
This notebook uses constraint programming to assign classes to groups and teachers, fulfilling the following restrictions:

* Each subject must be taught the specified number of weekly hours.
* A teacher cannot teach two classes at the same time.
* Teachers cannot exceed their maximum assigned weekly hours.
* The maximum number of hours per day for each subject and group is limited.
* If a subject is taught more than one hour per day in a group, the hours must be consecutive.

## Common definitions

In [ ]:
from ortools.sat.python import cp_model

# Create test data
class Subject:
    def __init__(self, id, name, course, weekly_hours=5, max_hours_per_day=1):
        self.id = id
        self.course = course
        self.name = name
        self.weekly_hours = weekly_hours
        self.max_hours_per_day = max_hours_per_day
    def __repr__(self):
        return f"{self.name} ({self.course})"

class Teacher:
    def __init__(self, id, name, max_hours_week=20, subjects=[]):
        self.id = id
        self.name = name
        self.max_hours_week = max_hours_week
        self.subjects = subjects
    def __repr__(self):
        return f"{self.name}, {self.max_hours_week} h/w, {[s.id for s in self.subjects]}"

## Timetable, Subjects and Teachers

In [ ]:
# Configure the time table
num_days = 5
num_hours = 5

all_groups = ["1-A", "1-B"]

# Subjects
math1 = Subject(id="math1", name="Maths", course="1", weekly_hours=10, max_hours_per_day=2)
all_subjects = [math1]


# Teachers
John = Teacher(id=1, name="John", max_hours_week=25, subjects=[math1])
all_teachers = [John]

print("Groups:", all_groups)
print("Subjects:", [repr(s) for s in all_subjects])
print("Teachers:", [repr(t) for t in all_teachers])

## Model & decission variables

In [ ]:
# Initialize model
model = cp_model.CpModel()

# Create decision variables (group-subject-teacher-day-hour)
assignments = {}  
for group in all_groups:
    course = group.split('-')[0]
    for subject in all_subjects:
        if subject.course == course:
            for teacher in all_teachers:
                if subject in teacher.subjects:
                    for d in range(num_days):
                        for h in range(num_hours):
                            key = (group, subject.id, teacher.id, d, h)
                            assignments[key] = model.NewBoolVar(f"g:{group} sub:{subject.id} t:{teacher.name} d:{d} h:{h}")


## Restrictions

In [ ]:

# Each subject must be taught the specified weekly hours
for group in all_groups:
    course = group.split('-')[0]
    for subject in all_subjects:
        if subject.course == course:
            model.Add(sum(assignments[key] for key in assignments if key[0] == group and key[1] == subject.id) == subject.weekly_hours)

# A teacher cannot teach two classes at the same time
for teacher in all_teachers:
    for d in range(num_days):
        for h in range(num_hours):
            model.AddAtMostOne(assignments[key] for key in assignments if key[2] == teacher.id and key[3] == d and key[4] == h)

# Teachers cannot exceed their maximum assigned weekly hours.
for teacher in all_teachers:
    max_hours = teacher.max_hours_week
    teacher_total_hours = sum(assignments[key] for key in assignments if key[2] == teacher.id)
    model.Add(teacher_total_hours <= max_hours)
    
# The maximum number of hours per day for each subject and group is limited.
for group in all_groups:
    course = group.split('-')[0]
    for subject in all_subjects:
        if subject.course == course:
            for teacher in all_teachers:
                if subject in teacher.subjects:
                    for d in range(num_days):
                        # Get hours for this specific group-subject-teacher combination
                        hour_vars = [assignments[key] for key in assignments  if key[0] == group and key[1] == subject.id and key[2] == teacher.id and key[3] == d]
                        model.Add(sum(hour_vars) <=subject.max_hours_per_day)

The following cell is commented because it is not working as expected

In [ ]:
# # If a subject is taught more than one hour per day in a group, the hours must be consecutive.

# for group in all_groups:
#     course = group.split('-')[0]
#     for subject in all_subjects:
#         if subject.course == course:
#             for d in range(num_days):
#                 # For each hour, create a variable indicating if there's a class or not
#                 hour_vars = []
#                 for h in range(num_hours):
#                     # Find all possible assignments for this hour
#                     vars_at_hour = [assignments[key] for key in assignments 
#                                     if key[0] == group and key[1] == subject.id 
#                                     and key[3] == d and key[4] == h]
#                     if vars_at_hour:
#                         # If there are possible assignments, create variable representing "there's a class at this hour"
#                         hour_var = model.NewBoolVar(f"class_{subject.id}_{group}_{d}_{h}")
#                         model.AddBoolOr(vars_at_hour).OnlyEnforceIf(hour_var)
#                         model.AddBoolAnd([v.Not() for v in vars_at_hour]).OnlyEnforceIf(hour_var.Not())
#                         hour_vars.append(hour_var)

#                 if len(hour_vars) > 1:
#                     # For each non-adjacent pair of hours
#                     for h1 in range(len(hour_vars)):
#                         for h2 in range(h1 + 2, len(hour_vars)):  # h2 > h1 + 1
#                             # Create variable for this constraint
#                             consecutive = model.NewBoolVar(f"{subject.id}:{group}:{d}:{h1}-{h2}")
                            
#                             # If h1 and h2 are assigned
#                             model.AddBoolAnd([hour_vars[h1], hour_vars[h2]]).OnlyEnforceIf(consecutive)

#                             # Then all intermediate hours must be assigned
#                             middle_hours = [hour_vars[h] for h in range(h1 + 1, h2)]
#                             if middle_hours:
#                                 model.AddBoolAnd(middle_hours).OnlyEnforceIf(consecutive)

#                             # Force the constraint to be satisfied
#                             model.Add(consecutive == 1)


## Solving the problem

In [ ]:
# Solve the model and display the timetable
from IPython.display import display, Markdown

solver = cp_model.CpSolver()
status = solver.Solve(model)

# Create and display markdown timetable
if status == cp_model.FEASIBLE or status == cp_model.OPTIMAL:
    for group in all_groups:
        course = group.split('-')[0]
        markdown_text = [f"## Schedule for {group}\n"]
        markdown_text.append("| Hour | Monday | Tuesday | Wednesday | Thursday | Friday |")
        markdown_text.append("|------|---------|----------|-----------|-----------|---------|")
        
        for h in range(num_hours):
            row = [f"Hour {h}"]
            for d in range(num_days):
                cell_content = ""
                for subject in all_subjects:
                    if subject.course == course:
                        for teacher in all_teachers:
                            if subject in teacher.subjects:
                                key = (group, subject.id, teacher.id, d, h)
                                if solver.Value(assignments[key]) == 1:
                                    cell_content = f"{subject.name}<br>{teacher.name}"
                row.append(cell_content if cell_content else "-")
            markdown_text.append(f"| {' | '.join(row)} |")
        
        # Display the timetable as rendered markdown
        display(Markdown('\n'.join(markdown_text)))
else:
    print("No feasible solution found.")